In [7]:
import pandas as pd
from sklearn.naive_bayes import BernoulliNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from pymorphy2 import MorphAnalyzer
from nltk import word_tokenize

In [8]:
# данные
test = pd.read_csv('dataset/test.tsv', sep='\t')
train = pd.read_csv('dataset/train.tsv', sep='\t')

test.head(10)

,title,is_fake
0,Роскомнадзор представил реестр сочетаний цвето...,0
1,Ночью под Минском на президентской горе Белара...,0
2,Бывший спичрайтер Юрия Лозы рассказал о трудно...,0
3,"Сельская церковь, собравшая рекордно низкое ко...",0
4,Акции Google рухнули после объявления о переза...,0
5,Курс доллара вырос до исторического максимума,0
6,ОПЕК назвала оптимальный уровень цен на нефть,0
7,Российская авиакомпания откроет рейсы в Тбилис...,0
8,Швейцарская горнолыжница расстреляна в доме ро...,0
9,Учреждена театральная премия имени Гарольда Пи...,0


In [9]:
class LemmaTokenizer:
    def __init__(self):
        self.lemmatizer = MorphAnalyzer()
    def __call__(self, doc):
        return [self.lemmatizer.normal_forms(t)[0] for t in word_tokenize(doc)]
    
bernoulli_pipe = Pipeline(steps=[('vectorizer', CountVectorizer(tokenizer=LemmaTokenizer(), binary=True, token_pattern=None)),
                                 ('classifier', BernoulliNB(alpha=0.35))])
texts = train.title.to_numpy()
targets = train.is_fake.to_numpy()

new_texts = test.title.to_numpy()

In [10]:
# обучение и предсказание
bernoulli_pipe.fit(texts, targets)
predicted = bernoulli_pipe.predict(new_texts)

In [11]:
test.is_fake = predicted
test.to_csv('predictions.tsv',  sep='\t', index=False)